In [2]:
# Imports
import pandas as pd
from collections import Counter

In [16]:
# Transferring the csv into a pandas dataframe, check for a successful transfer
review_df = pd.read_csv('metrocar_review.csv')
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49999 entries, 0 to 49998
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  49999 non-null  int64 
 1   year     49999 non-null  int64 
 2   month    49999 non-null  int64 
 3   hour     49999 non-null  int64 
 4   rating   49999 non-null  int64 
 5   review   49999 non-null  object
dtypes: int64(5), object(1)
memory usage: 2.3+ MB


In [17]:
# Defining our key words 
# I went for a supervised learning approach, as there were aspects of the reviews I wanted to explore for our funnel
# Within these aspects there are key words that would correspond to a positive, satisfactory or negative review


app_keywords = '|'.join(['taps', 'pickup location', 'requesting', 'app', 'application', 'mobile app', 'user', 'app interface', 'GPS', 'navigation', 'estimated time of arrival'])
app_sentiments = {
    'positive': '|'.join(['unmatched', 'go-to', 'user_friendly', 'convenience', 'easy']),
    'satisfactory': '|'.join(['could be better', 'glitches', 'could have been more precise', 'could be more accurate', 'decent', 'was not accurate']),
    'negative': '|'.join(['constant glitches', 'incorrect arrival time', 'incorrect arrival times', 'crashed repeatedly'])
}

driver_keywords = '|'.join(['taxi', 'speed limit', 'navigation skills', 'drivers', 'car', 'cars'])
driver_sentiments = {
    'positive': '|'.join(['courteous', 'polite', 'professional', 'knowledgeable', 'punctual', 'friendly']),
    'satisfactory': '|'.join(['could have been better', 'communication problems']),
    'negative': '|'.join(['unfamiliar', 'got lost', 'a bit unfriendly', 'rude', 'difficult', 'recklessly', 'disinterested', 'explanation', 'inexperienced', 'poor', 'unprofessional', 'disrespectful', 'distracted', 'disgusted', 'horrible'])
}

ride_keywords = '|'.join(['pickup', 'vehicle', 'service', 'arrived', 'ride', 'transportation', 'trip', 'journey', 'Metrocar', 'vehicle', 'smell', 'odor', 'condition'])
ride_sentiments = {
    'positive': '|'.join(['on time', 'highly', 'good', 'smooth', 'hassle-free', 'go-to', 'Great service!', 'efficient', 'safe', 'promptly', 'reliable', 'convenience', 'unmatched', 'comfortable', 'clean']),
    'satisfactory': '|'.join(['fine', 'longer', 'would be helpful']),
    'negative': '|'.join(['weird', 'strange', 'not as clean', 'not in the best condition', 'regrettable', 'uncomfortable', 'unpleasant', 'not clean', 'not a great', 'disgusted', 'broke down', 'unsafe', 'wrong', 'terrible', 'worst'])
}

price_keywords = '|'.join(['fare', 'price', 'overcharged', 'money'])
price_sentiments = {
    'positive': '|'.join(['saves', 'quickly', 'affordable']),
    'satisfactory': '|'.join(['increased', 'higher than expected']),
    'negative': '|'.join(['overcharged'])
}

wait_time_keywords = '|'.join(['wait time', 'route', 'arrival', 'canceled', 'experience'])
wait_time_sentiments = {
    'positive': '|'.join(['promptly']), 
    'satisfactory': '|'.join(['preferred']),
    'negative': '|'.join(['stranded', 'late', 'incorrect', 'wrong', 'last minute'])
}

aspects = {
    app_keywords: 'app_keywords',
    driver_keywords: 'driver_keywords',
    price_keywords: 'price_keywords',
    ride_keywords: 'ride_keywords',
    wait_time_keywords: 'wait_time_keywords'
}

In [19]:
# Now using our lists above, we will write a for loop that will check which aspect we are looking at
# Within the loop, there is an if / elif statement to make sure we are going through the correct list of key words 
# There are keywords that could be linked to any aspect
# To get around this, the loop checks for the a mention of the aspect, then checks for a keyword
# This determines whether a value of 1 (True) or 0 (False) will be given
# I wanted a column for each aspect and each keyword, so we can use these columns with flexibility in tableau

for aspect in aspects: 
    if aspect == app_keywords:
        for sentiment, sentiment_keywords in app_sentiments.items():
            column_name = f'app_{sentiment}'  
            review_df[column_name] = review_df['review'].str.contains(sentiment_keywords, case=False, regex=True)

            review_df[column_name] = review_df[column_name].astype(int)
    
    elif aspect == driver_keywords:
        for sentiment, sentiment_keywords in driver_sentiments.items():
            column_name = f'driver_{sentiment}'  
            review_df[column_name] = review_df['review'].str.contains(sentiment_keywords, case=False, regex=True)

            review_df[column_name] = review_df[column_name].astype(int)
            
    elif aspect == ride_keywords:
        for sentiment, sentiment_keywords in ride_sentiments.items():
            column_name = f'ride_{sentiment}'
            review_df[column_name] = review_df['review'].str.contains(sentiment_keywords, case=False, regex=True)

            review_df[column_name] = review_df[column_name].astype(int)
            
    elif aspect == price_keywords:
        for sentiment, sentiment_keywords in price_sentiments.items():
            column_name = f'price_{sentiment}'  
            review_df[column_name] = review_df['review'].str.contains(sentiment_keywords, case=False, regex=True)

            review_df[column_name] = review_df[column_name].astype(int)
    

    elif aspect == wait_time_keywords:
        for sentiment, sentiment_keywords in wait_time_sentiments.items():
            column_name = f'wait_time_{sentiment}' 
            review_df[column_name] = review_df['review'].str.contains(sentiment_keywords, case=False, regex=True)

            review_df[column_name] = review_df[column_name].astype(int)

review_df.head()


,user_id,year,month,hour,rating,review,app_positive,app_satisfactory,app_negative,driver_positive,...,driver_negative,price_positive,price_satisfactory,price_negative,ride_positive,ride_satisfactory,ride_negative,wait_time_positive,wait_time_satisfactory,wait_time_negative
0,114117,2021,11,17,4,Solid service with a few improvements needed. ...,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1,102818,2021,6,9,4,The driver was friendly and the car was comfor...,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
2,103664,2021,6,16,2,Regrettable ride with Metrocar. The driver was...,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,109644,2021,8,11,3,Had some hiccups with Metrocar. The driver can...,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,103040,2021,3,8,4,Mostly satisfied with Metrocar. The driver was...,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0


In [20]:
def map_values(row):
    if row['app_positive'] == 1:
        return 'positive', 1
    elif row['app_satisfactory'] == 1:
        return 'satisfactory', 1
    elif row['app_negative'] == 1:
        return 'negative', 1
    else:
        return '', 0
    
def map_driver_values(row):
    if row['driver_positive'] == 1:
        return 'positive', 1
    elif row['driver_satisfactory'] == 1:
        return 'satisfactory', 1
    elif row['driver_negative'] == 1:
        return 'negative', 1
    else:
        return '', 0

def map_ride_values(row):
    if row['ride_positive'] == 1:
        return 'positive', 1
    elif row['ride_satisfactory'] == 1:
        return 'satisfactory', 1
    elif row['ride_negative'] == 1:
        return 'negative', 1
    else:
        return '', 0
    
def map_price_values(row):
    if row['price_positive'] == 1:
        return 'positive', 1
    elif row['price_satisfactory'] == 1:
        return 'satisfactory', 1
    elif row['price_negative'] == 1:
        return 'negative', 1
    else:
        return '', 0
    
def map_wait_time_values(row):
    if row['wait_time_positive'] == 1:
        return 'positive', 1
    elif row['wait_time_satisfactory'] == 1:
        return 'satisfactory', 1
    elif row['wait_time_negative'] == 1:
        return 'negative', 1
    else:
        return '', 0


# Apply each function to create a keyword columns and a value column
review_df['app'], review_df['app_val'] = zip(*review_df.apply(map_values, axis=1))
review_df['driver'], review_df['driver_val'] = zip(*review_df.apply(map_driver_values, axis=1))
review_df['ride'], review_df['ride_val'] = zip(*review_df.apply(map_ride_values, axis=1))
review_df['price'], review_df['price_val'] = zip(*review_df.apply(map_price_values, axis=1))
review_df['wait_time'], review_df['wait_time_val'] = zip(*review_df.apply(map_wait_time_values, axis=1))

In [7]:
review_df.head()

,user_id,year_month,hour_range,rating,review,app_positive,app_satisfactory,app_negative,driver_positive,driver_satisfactory,...,app,app_val,driver,driver_val,ride,ride_val,price,price_val,wait_time,wait_time_val
0,117229,2022 02,08:00,1,Unsafe ride with Metrocar. The driver was dist...,0,0,0,0,0,...,,0,negative,1,positive,1,,0,,0
1,106358,2021 06,19:00,1,"A nightmare experience. The driver was rude, a...",0,0,0,0,0,...,,0,negative,1,negative,1,,0,,0
2,100291,2021 09,08:00,4,The driver was friendly and the car was comfor...,0,1,0,1,0,...,satisfactory,1,positive,1,positive,1,,0,,0
3,102509,2021 03,17:00,1,Absolutely disgusted with Metrocar. The driver...,0,0,0,0,0,...,,0,negative,1,negative,1,negative,1,,0
4,110021,2021 08,09:00,3,Average experience with Metrocar. The driver a...,0,0,0,0,0,...,,0,,0,positive,1,,0,,0


In [22]:
# Looking at the first five rows determines if the code works but now we will check the total count for each column

aspects = ['app', 'driver', 'ride', 'price', 'wait_time']

for aspect in aspects:
    print(f'{aspect.capitalize()}:')
    aspect_columns = [f'{aspect}_{sentiment}' for sentiment in app_sentiments.keys()]
    for column in aspect_columns:
        print(f'{column}: {review_df[column].value_counts()}')
    print()


App:
app_positive: app_positive
0    45699
1     4300
Name: count, dtype: int64
app_satisfactory: app_satisfactory
0    42048
1     7951
Name: count, dtype: int64
app_negative: app_negative
0    47528
1     2471
Name: count, dtype: int64

Driver:
driver_positive: driver_positive
0    33570
1    16429
Name: count, dtype: int64
driver_satisfactory: driver_satisfactory
0    48219
1     1780
Name: count, dtype: int64
driver_negative: driver_negative
0    32459
1    17540
Name: count, dtype: int64

Ride:
ride_positive: ride_positive
1    29431
0    20568
Name: count, dtype: int64
ride_satisfactory: ride_satisfactory
0    42294
1     7705
Name: count, dtype: int64
ride_negative: ride_negative
0    36393
1    13606
Name: count, dtype: int64

Price:
price_positive: price_positive
0    47468
1     2531
Name: count, dtype: int64
price_satisfactory: price_satisfactory
0    46424
1     3575
Name: count, dtype: int64
price_negative: price_negative
0    48586
1     1413
Name: count, dtype: int64

Wa

In [23]:
# Alright, we have the total count of each keyword and their aspects
# But what are people mentioning the most?
# The function counts the most occuring words in each keyword aspect 
# It then prints the string of the most occuring word

def count_words_in_reviews(dataFrame, sentiment_keywords, sentiment_name):
    word_counts = Counter()
    for review in dataFrame['review']:
        for word in sentiment_keywords.split('|'):
            count = review.lower().count(word)
            word_counts[word] += count

    print(f'TOP 5 occurrences in {sentiment_name}:')
    for word, count in word_counts.most_common(5):
        print(f'{word}: {count}')

aspects = {
    'App': {
        'Keywords': app_keywords,
        'Sentiments': app_sentiments,
    },
    'Driver': {
        'Keywords': driver_keywords,
        'Sentiments': driver_sentiments,
    },
    'Ride': {
        'Keywords': ride_keywords,
        'Sentiments': ride_sentiments,
    },
    'Price': {
        'Keywords': price_keywords,
        'Sentiments': price_sentiments,
    },
    'Wait Time': {
        'Keywords': wait_time_keywords,
        'Sentiments': wait_time_sentiments,
    }
}

In [24]:
try: 
    for aspect, data in aspects.items():
        print(f'{aspect}:')
        for sentiment, keywords in data['Sentiments'].items():
            count_words_in_reviews(review_df, keywords, sentiment)
        print()

except Exception as e: 
    print(f"""An Error occured: {e}""")

App:
TOP 5 occurrences in positive:
convenience: 1764
easy: 1308
unmatched: 1262
go-to: 1228
user_friendly: 0
TOP 5 occurrences in satisfactory:
glitches: 2363
decent: 1813
could have been more precise: 1308
was not accurate: 1235
could be more accurate: 1232
TOP 5 occurrences in negative:
crashed repeatedly: 1460
constant glitches: 536
incorrect arrival time: 475
incorrect arrival times: 475

Driver:
TOP 5 occurrences in positive:
professional: 6508
friendly: 5488
polite: 3798
courteous: 1806
punctual: 1282
TOP 5 occurrences in satisfactory:
could have been better: 1293
communication problems: 487
TOP 5 occurrences in negative:
rude: 3515
unprofessional: 3490
unfamiliar: 2795
poor: 2435
horrible: 1534

Ride:
TOP 5 occurrences in positive:
comfortable: 7242
reliable: 5195
good: 3902
smooth: 3814
clean: 3513
TOP 5 occurrences in satisfactory:
longer: 5108
fine: 1800
would be helpful: 1293
TOP 5 occurrences in negative:
uncomfortable: 3453
terrible: 2941
unsafe: 1433
broke down: 1415
wro

In [25]:
columns_to_drop = [
    'app_positive', 'app_satisfactory', 'app_negative',
    'driver_positive', 'driver_satisfactory', 'driver_negative',
    'price_positive', 'price_satisfactory', 'price_negative',
    'ride_positive', 'ride_satisfactory', 'ride_negative',
    'wait_time_positive', 'wait_time_satisfactory', 'wait_time_negative'
]

review_df.drop(columns=columns_to_drop, inplace=True)

In [30]:
# Finally we need to perform a group by so we can link these new columns to our funnel data
# I decided the best way to do this would be to group by the time components as we do not have a common key
# We're also going to be grouping by each keyword to reduce the number of rows 
# This format will be useful for tableau visualisations

grouped_df = review_df.groupby(['year','month', 'hour', 'app', 'driver', 'ride', 'price', 'wait_time']).agg({
    'app_val': 'sum',
    'driver_val': 'sum',
    'ride_val': 'sum',
    'price_val': 'sum',
    'wait_time_val': 'sum'
}).reset_index()

# Print the resulting DataFrame
grouped_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6415 entries, 0 to 6414
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year           6415 non-null   int64 
 1   month          6415 non-null   int64 
 2   hour           6415 non-null   int64 
 3   app            6415 non-null   object
 4   driver         6415 non-null   object
 5   ride           6415 non-null   object
 6   price          6415 non-null   object
 7   wait_time      6415 non-null   object
 8   app_val        6415 non-null   int64 
 9   driver_val     6415 non-null   int64 
 10  ride_val       6415 non-null   int64 
 11  price_val      6415 non-null   int64 
 12  wait_time_val  6415 non-null   int64 
dtypes: int64(8), object(5)
memory usage: 651.7+ KB


In [31]:
review_df.to_csv('review_wrangled_data.csv', index=False)